In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
from copy import copy

In [ ]:
def prep_default_info(matches_number, fighters_number):
    def prep_matches(matches):
        out = []
        for match in range(1, matches+1):
            out += [match, match]
        return out
    
    
    def random_opponent(fighter, num_fighters):
        new_number = random.randrange(1, num_fighters)
        if new_number != fighter:
            return new_number
        return random_opponent(fighter, num_fighters)

    def prep_fighters(matches_number, fighters_number):
        fighters = [random.randint(1, fighters_number) for match in range(matches_number)]
        out = []
        for fighter in fighters:
            matchup_a, matchup_b = fighter, random_opponent(fighter, fighters_number) 
            assert matchup_a != matchup_b
            out += [matchup_a, matchup_b]
        return out
    
    def prep_results(matches_number):
        result_a = [random.randint(0,2)/2 for match in range(matches_number)]
        out = []
        for result in result_a:
            out += [result, 1-result]
        return out
        
    
    matches = prep_matches(matches_number)
    fighters = prep_fighters(matches_number, fighters_number)
    results = prep_results(matches_number)
    data = {'match':  matches,
            'fighters': fighters,
            'results':  results}
    return data


class Elo(object):
    def __init__(self, data, k=32):
        """
            data = {'match':  matches,
                    'fighters': fighters,
                    'results':  results}
        """
        self.df = pd.DataFrame(data)
        fighter_list = list(set(self.df['fighters']))
        self.fighters = {fighter: 1500 for fighter in fighter_list}
        self.k = k
        self.set_all_elos()
        
    def get_opponent_id(self, match, fighter):
        df = self.df
        results = list(df[df['match'] == match]['fighters'])
        assert len(results) == 2, "three way fight??"
        assert fighter in results, "fighter not in results"
        for result in results:
            if result != fighter:
                return result
    
    def set_new_elos(self, info):
        for fighter in info:
            self.fighters[fighter['fighter']] = fighter['elo']
            
    def get_new_elos(self, rows):
        fighter_1, fighter_2 = [self.df.loc[row, 'fighters'] for row in rows]
        for fighter in [fighter_1, fighter_2]:
            try:
                self.add_fighter(fighter)
            except ValueError:
                pass
        result_1, result_2 = [self.df.loc[row, 'results'] for row in rows]
        old_elo_1, old_elo_2 = self.fighters[fighter_1], self.fighters[fighter_2] 
        elo_1 = self.calculate_single_elo(old_elo_1, old_elo_2, result_1)
        elo_2 = self.calculate_single_elo(old_elo_2, old_elo_1, result_2)
        
        fighter_1_info = {
            'fighter': fighter_1,
            'elo': elo_1,
            'old_elo': old_elo_1,
            'change': elo_1 - old_elo_1
            
            }
        fighter_2_info = copy(fighter_1_info)
        fighter_2_info['fighter'] = fighter_2
        fighter_2_info['elo'] = elo_2
        fighter_2_info['old_elo'] = old_elo_2
        fighter_2_info['change'] = elo_2 - old_elo_2
        info = [fighter_1_info, fighter_2_info]
        return info
    
    def calculate_single_elo(self, original_fighter_elo, original_opponent_elo, result):
        transformed_ratings = list(map(lambda x: 10**(x/400), [original_fighter_elo, original_opponent_elo]))
        expected_score = transformed_ratings[0] / sum(transformed_ratings)
        return round(original_fighter_elo + self.k*(result - expected_score))
                                                            
    
    def set_match_elos(self, match_id):
        right_match = self.df['match'] == match_id
        row_a, row_b = self.df[right_match].index
        info = self.get_new_elos((row_a, row_b))
        
        new_elos = [fighter['elo'] for fighter in info]
        change = [fighter['change'] for fighter in info]
        self.df.loc[row_a:row_b, 'elo'] = new_elos  # Actually setting ELOs
        self.df.loc[row_a:row_b, 'change'] = change  # Actually setting ELOs
        self.set_new_elos(info)

        
    def set_all_elos(self):
        for match in set(self.df['match']):
            self.set_match_elos(match)
        return self.df
    
    def add_fighter(self, fighter_id):
        if fighter_id in self.fighters:
            raise ValueError("Duplicate fighter") # Is this the right type of exception to raise?
        self.fighters[fighter_id] = 1500
    
    def prep_fight_rows_info(self, a, b, a_result):
        fight_number = self.df['match'].max() + 1
        a_info = {
            'fighters': a,
             'match': fight_number,
             'results': a_result
        }
        
        b_info = copy(a_info)
        b_info['fighters'] = b
        b_info['results'] = 1 - a_result
        return [a_info, b_info]
    
    def add_fight(self, a, b, a_result):
        row_info = self.prep_fight_rows_info(a, b, a_result)
        new_rows = pd.DataFrame(row_info)
        self.df = self.df.append(new_rows, ignore_index=True)
        fight_id = row_info[0]['match']
        self.set_match_elos(fight_id) # Seems a waste here to set like this - I have all the info I need without the whole df...

    
class Data(object):
    def __init__(self, elo):
        if type(elo) == dict:
            elo = Elo(data)
        self.elo = elo
        self.df = self.elo.df
        
        self.fighter_list = list(self.elo.fighters.keys())
        
    def fighter_record(self, fighter):
        fighter_rows = self.df['fighters'] == fighter
        matches = self.df[fighter_rows]['match'].values
        right_match = self.df['match'].isin(matches)
        not_fighter = self.df['fighters'] != fighter
        fighter_elo = self.df.loc[fighter_rows,'elo']
        fighter_elo.name = ('fighter_elo_after')
        fighter_elo.reset_index(drop=True, inplace=True)
        new_df = self.df.loc[right_match & not_fighter]
        new_df.reset_index(drop=True, inplace=True)
        new_df = pd.concat([new_df, fighter_elo], axis=1)
        new_df.rename(columns={'fighters':'opponent', 'elo':'opponent_elo_after', 'results':'fighter_result'}, inplace=True)
        new_df['fighter_result'] = new_df['fighter_result'].apply(lambda x: 1-x)
        return new_df
    
    
    def select_goats(self, n):
        try:
            self.goats
        except AttributeError:
            self.set_goats()
        if not n:
            n = len(self.goats)
        goat_ids = [fighter[0] for fighter in self.goats[:n]]
        return goat_ids

    
    def plot_goat_elos(self, n=0):
        goat_ids = self.select_goats(n)
        df = self.df
        df = df[df['fighters'].isin(goat_ids)]
        groupby_elos = df.groupby('fighters')['elo']
        
        # Plotting seemingly randomly gives the following warning
        # UserWarning: Attempting to set identical left==right results
        # in singular transformations; automatically expanding.
        # left=0.0, right=0.0
        #   'left=%s, right=%s') % (left, right))
        return groupby_elos.plot(legend=True) # Why does Legend not work here?
    
    def fighter_best(self, fighter_id):
        fighter = self.df['fighters'] == fighter_id
        one_max = self.df[fighter]['elo'].max()
        return one_max
    
    def get_match_indexes(self, match):
        return self.df[self.df['match'] == match].index
    
    def set_goats(self):
        fighter_prep = [(fighter, self.fighter_best(fighter)) for fighter in self.fighter_list]
        sorted_fighters = sorted(fighter_prep, key=lambda x: x[1], reverse=True)
        self.goats = sorted_fighters
    
    def get_opponent_fight_info(self, match_row):
        indexes = self.get_match_indexes(match_row['match'])
        for index in indexes:
            if index != match_row.name:
                return index
        raise Exception("Didnt find an opponent...?")
        
    def biggest_upsets(self, n=5):
        sorted_changes = self.df.sort_values(['change', 'elo'], ascending=False)
        biggest_winners = sorted_changes.head(n)
        out = biggest_winners.copy()
        out['opponent_info'] = out.apply(self.get_opponent_fight_info, axis=1)
        # add information on opponent 
        return out

In [ ]:
data = prep_default_info(200, 10)
y = Elo(data)
d = Data(y)

In [ ]:
d.fighter_record(4)